In [9]:
print("womp womp")

womp womp
time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [10]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [11]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Failure Prediction\heart.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [12]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

def ExerciseAngina_map(x):
    return x.map({'Y' : 1, 'N' : 0})

def ST_Slope_map(x):
    return x.map({'Down' : -1, 'Flat' : 0, 'Up' : 1})


def ChestPainType_map(x):
    return x.map({'ATA' : 1, 'NAP' : 0.5, 'ASY' : -0.5, 'TA' : -1})

def RestingECG_map(x):
    return x.map({'LVH' : -1, 'Normal' : 0, 'ST' : 1})

time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [13]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['ExerciseAngina'] = heart_attack[['ExerciseAngina']].apply(ExerciseAngina_map)
heart_attack['ST_Slope'] = heart_attack[['ST_Slope']].apply(ST_Slope_map)
heart_attack['ChestPainType'] = heart_attack[['ChestPainType']].apply(ChestPainType_map)
heart_attack['RestingECG'] = heart_attack[['RestingECG']].apply(RestingECG_map)
# heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
# heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
# heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
# heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

# unique_sex = heart_attack['Sex'].unique()
# print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)

# heart_attack.head()

time: 15 ms (started: 2023-12-15 11:09:12 -05:00)


In [14]:
# mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
#                 'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

# mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

# median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# # Fill NaN values with the mode of the column for mode_columns
# for col in mode_columns:
#     heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# # Fill NaN values with the median of the column for median_columns
# for col in median_columns:
#     heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# # Fill NaN values with the mean of the column for mean_columns
# for col in mean_columns:
#     heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)


# unique_sex = heart_attack['Sex'].unique()
# print(unique_sex)
# unique = heart_attack['Hemisphere'].unique()
# print(unique)
# unique_country = heart_attack['Country'].unique()
# print(unique_country)
# unique_diet = heart_attack['Diet'].unique()
# print(unique_diet)
# unique_continent = heart_attack['Continent'].unique()
# print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()

Columns with NaN values after fillna operations:
[]


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1.0,140,289,0,0,172,0,0.0,1,0
1,49,0,0.5,160,180,0,0,156,0,1.0,0,1
2,37,1,1.0,130,283,0,1,98,0,0.0,1,0
3,48,0,-0.5,138,214,0,0,108,1,1.5,0,1
4,54,1,0.5,150,195,0,0,122,0,0.0,1,0


time: 16 ms (started: 2023-12-15 11:09:12 -05:00)


In [15]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [16]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:10].values
Y = heart_attack.iloc[:, 11].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

# Convert the numpy arrays into tensors
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
time: 15 ms (started: 2023-12-15 11:09:12 -05:00)


In [17]:
# Defining PCA function
def PCA_function(X_train, X_val, K):
  pca = PCA(n_components=K)
  X_train_PCA = pca.fit_transform(X_train)
  X_val_PCA = pca.fit_transform(X_val)
  return X_train_PCA, X_val_PCA

time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [18]:
(X_train_PCA, X_val_PCA) = PCA_function(X_train, X_val, 2)

print("The shape of X_train after PCA is", X_train_PCA.shape)
print("The shape of X_val after PCA is", X_val_PCA.shape)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train_PCA = scale.fit_transform(X_train_PCA)
X_val_PCA = scale.fit_transform(X_val_PCA)

X_train_PCA_t = torch.tensor(X_train_PCA, dtype=torch.float)
X_val_PCA_t = torch.tensor(X_val_PCA, dtype=torch.float)

print("\nThe shape of the X_train tensor is", X_train_PCA_t.shape)
print("The shape of the X_val tensor is", X_val_PCA_t.shape)

The shape of X_train after PCA is (734, 2)
The shape of X_val after PCA is (184, 2)

The shape of the X_train tensor is torch.Size([734, 2])
The shape of the X_val tensor is torch.Size([184, 2])
time: 16 ms (started: 2023-12-15 11:09:12 -05:00)


In [19]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 0 ns (started: 2023-12-15 11:09:12 -05:00)


In [20]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_PCA_t,
    val_t_in = X_val_PCA_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 21.6165, 	Validation loss 17.4866
Epoch 2, 	Training loss 20.5279, 	Validation loss 16.6155
Epoch 3, 	Training loss 19.4978, 	Validation loss 15.7913
Epoch 500, 	Training loss 0.3871, 	Validation loss 0.4637
Epoch 1000, 	Training loss 0.3155, 	Validation loss 0.3772
Epoch 1500, 	Training loss 0.2807, 	Validation loss 0.3300
Epoch 2000, 	Training loss 0.2616, 	Validation loss 0.3018
Epoch 2500, 	Training loss 0.2506, 	Validation loss 0.2840
Epoch 3000, 	Training loss 0.2441, 	Validation loss 0.2724
Epoch 3500, 	Training loss 0.2402, 	Validation loss 0.2645
Epoch 4000, 	Training loss 0.2378, 	Validation loss 0.2592
Epoch 4500, 	Training loss 0.2364, 	Validation loss 0.2554
Epoch 5000, 	Training loss 0.2355, 	Validation loss 0.2528


tensor([ 0.2330,  0.0965, -0.0529,  0.4282], requires_grad=True)

time: 1.48 s (started: 2023-12-15 11:09:12 -05:00)


In [21]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")


Training Accuracy: 0.5654
Training Precision: 0.5814
Training Recall: 0.5773
Training F1 Score: 0.5793

Validation Accuracy: 0.5054
Validation Precision: 0.5264
Validation Recall: 0.5256
Validation F1 Score: 0.5260
time: 0 ns (started: 2023-12-15 11:09:14 -05:00)


In [22]:
#Sequential Model

model_seq = nn.Sequential(
    nn.Linear(2, 1024),
    nn.Tanh(),
    nn.Linear(1024, 2048),
    nn.Tanh(),
    nn.Linear(2048, 512),
    nn.Tanh(),
    nn.Linear(512, 1)
            )
model_seq

Sequential(
  (0): Linear(in_features=2, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 16 ms (started: 2023-12-15 11:09:14 -05:00)


In [23]:
sum([p.numel() for p in model_seq.parameters()])

3151873

time: 0 ns (started: 2023-12-15 11:09:14 -05:00)


In [24]:
#Let's define our training loop that will be used

def training_loop_seq(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 0 ns (started: 2023-12-15 11:09:14 -05:00)


In [25]:
#Let's try doing some training

optimizer_1 = optim.Adam(model_seq.parameters(), lr=1e-4)

training_loop_seq(
    n_epochs = 501,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_PCA_t,
    x_val = X_val_PCA_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.6835, Validation loss: 0.7208
Epoch: 10, Training loss: 0.2903, Validation loss: 0.3060
Epoch: 20, Training loss: 0.2105, Validation loss: 0.2033
Epoch: 30, Training loss: 0.1984, Validation loss: 0.1947
Epoch: 40, Training loss: 0.1966, Validation loss: 0.1988
Epoch: 50, Training loss: 0.1906, Validation loss: 0.1908
Epoch: 60, Training loss: 0.1893, Validation loss: 0.1889
Epoch: 70, Training loss: 0.1886, Validation loss: 0.1897
Epoch: 80, Training loss: 0.1880, Validation loss: 0.1903
Epoch: 90, Training loss: 0.1876, Validation loss: 0.1906
Epoch: 100, Training loss: 0.1872, Validation loss: 0.1906
Epoch: 110, Training loss: 0.1869, Validation loss: 0.1904
Epoch: 120, Training loss: 0.1866, Validation loss: 0.1902
Epoch: 130, Training loss: 0.1863, Validation loss: 0.1900
Epoch: 140, Training loss: 0.1861, Validation loss: 0.1899
Epoch: 150, Training loss: 0.1858, Validation loss: 0.1899
Epoch: 160, Training loss: 0.1856, Validation loss: 0.1899
Epoch: 1

In [26]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positives = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

time: 15 ms (started: 2023-12-15 11:09:52 -05:00)


In [27]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the validation set
with torch.no_grad():
    train_predictions_val = torch.round(torch.sigmoid(model_seq(X_val_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the validation set
accuracy_val = calculate_accuracy(train_predictions_val, Y_val_t)
recall_val = calculate_recall(train_predictions_val, Y_val_t)
precision_val = calculate_precision(train_predictions_val, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the calculated metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.5490
Training Recall: 1.0000
Training Precision: 0.5478
Training F1 Score: 0.7079

Validation Accuracy: 0.5978
Validation Recall: 1.0000
Validation Precision: 0.5912
Validation F1 Score: 0.7431
time: 47 ms (started: 2023-12-15 11:09:52 -05:00)


In [28]:
model_fnn = nn.Sequential(
            nn.Linear(2, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 16 ms (started: 2023-12-15 11:09:52 -05:00)


In [29]:
sum([p.numel() for p in model_fnn.parameters()])

4758145

time: 0 ns (started: 2023-12-15 11:09:52 -05:00)


In [30]:
learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 1501

#This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_PCA_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.699251
Epoch: 10, Loss: 0.585557
Epoch: 20, Loss: 0.578067
Epoch: 30, Loss: 0.584803
Epoch: 40, Loss: 0.580392
Epoch: 50, Loss: 0.604798
Epoch: 60, Loss: 0.568092
Epoch: 70, Loss: 0.572756
Epoch: 80, Loss: 0.571979
Epoch: 90, Loss: 0.565025
Epoch: 100, Loss: 0.574840
Epoch: 110, Loss: 0.569844
Epoch: 120, Loss: 0.570944
Epoch: 130, Loss: 0.563022
Epoch: 140, Loss: 0.558242
Epoch: 150, Loss: 0.562389
Epoch: 160, Loss: 0.565654
Epoch: 170, Loss: 0.560983
Epoch: 180, Loss: 0.559264
Epoch: 190, Loss: 0.559316
Epoch: 200, Loss: 0.556369
Epoch: 210, Loss: 0.557863
Epoch: 220, Loss: 0.561548
Epoch: 230, Loss: 0.556681
Epoch: 240, Loss: 0.561481
Epoch: 250, Loss: 0.560376
Epoch: 260, Loss: 0.555600
Epoch: 270, Loss: 0.558778
Epoch: 280, Loss: 0.560203
Epoch: 290, Loss: 0.557231
Epoch: 300, Loss: 0.559874
Epoch: 310, Loss: 0.552773
Epoch: 320, Loss: 0.555220
Epoch: 330, Loss: 0.554105
Epoch: 340, Loss: 0.556662
Epoch: 350, Loss: 0.555707
Epoch: 360, Loss: 0.555429
Epoch: 370, 

In [31]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_PCA_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.453678
Training Precision: 0.226839
Training Recall: 0.500000
Training F1 score: 0.312090
time: 46 ms (started: 2023-12-15 11:12:24 -05:00)


In [32]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_PCA_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
recall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Training Precision: %f" % precision_val)
print("Training Recall: %f" % recall_val)
print("Training F1 score: %f" % F1_score_val)

Validation Accuracy: 0.418478
Training Precision: 0.209239
Training Recall: 0.500000
Training F1 score: 0.295019
time: 16 ms (started: 2023-12-15 11:12:24 -05:00)
